In [1]:
from bs4 import BeautifulSoup
import urllib.request
import urllib.parse
import pandas as pd

# Molon 주간 Top 100구하기

In [2]:
import requests
url = 'https://www.melon.com/chart/week/index.htm'
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Trident/7.0; rv:11.0) like Gecko'}
req = requests.get(url, headers = header)
html= req.text

In [3]:
# #하나만 가져오기: 태그 객체 = select_one(CSS Selector 문자열) #CSS Selector: 태그의 계층 구조상의 위치를 설명
soup = BeautifulSoup(html, 'html.parser')

In [4]:
soup.select_one('.yyyymmdd').get_text() # 안의 내용물만 안나올때 strip

'\r\n\t\t\t\t\t\t\t2020.11.09 ~ 2020.11.15\r\n\t\t\t\t\t\t'

In [5]:
date = soup.select_one('.yyyymmdd').get_text().strip()
date

'2020.11.09 ~ 2020.11.15'

In [6]:
start_day = date[:10]
end_day = date[13:]
start_day, end_day

('2020.11.09', '2020.11.15')

### 곡 리스트 찾기 

In [7]:
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')
len(trs)

100

In [27]:
print(trs)

 class="button_icons scrap" onclick="melon.play.addPlayList('32034629');" title="담기" type="button"><span class="none">담기</span></button>
</div></td>
<td><div class="wrap t_center">
<button class="button_icons download" onclick="melon.buy.goBuyProduct('frm', '32034629', '3C0001', '','0', '19041301');" title="다운로드" type="button"><span class="none">다운로드</span></button>
</div></td>
<td><div class="wrap t_center">
<button class="button_icons video" onclick="melon.link.goMvDetail('19041301', '32034629','song');" title="뮤직비디오" type="button"><span class="none">뮤직비디오</span></button>
</div></td>
</tr>, <tr class="lst100" data-song-no="32955381" id="lst100" style="">
<td><div class="wrap t_right"><input class="input_check" name="input_check" title="밤하늘의 저 별처럼 곡 선택" type="checkbox" value="32955381"/></div></td>
<td><div class="wrap t_center"><span class="rank">95</span><span class="none">위</span></div></td>
<!-- 차트순위 추가 -->
<td><div class="wrap">
<span class="rank_wrap" title="4단계 하락">
<span class

In [8]:
rank = int(trs[0].select_one('.rank').string)
rank

1

In [9]:
for i in [0, 1, 2, 19]:
    entry = trs[i].select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    print(diff)

0
1
-1
-100


In [10]:
trs[0].select_one('.rank_wrap')

<span class="rank_wrap" title="순위 동일">
<span class="bullet_icons rank_static"><span class="none">순위 동일</span></span>
<span class="none">0</span>
</span>

In [11]:
info = trs[0].select_one('.ellipsis.rank01')
title= info.find('a').string
info = trs[0].select_one('.ellipsis.rank02')
singer = info.find('a').string
info = trs[0].select_one('.ellipsis.rank03')
album = info.find('a').string
title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

In [18]:
print(info)

<div class="ellipsis rank03">
<a href="javascript:melon.link.goAlbumDetail('10515548');" title="너의 하루 (바른연애 길잡이 X 로꼬) - 페이지 이동">너의 하루 (바른연애 길잡이 X 로꼬)</a>
</div>


In [12]:
info = trs[0].select('.ellipsis')
title = info[0].find('a').string
singer = info[1].find('a').string
album = info[2].find('a').string
title, singer, album

('Dynamite', '방탄소년단', 'Dynamite (DayTime Version)')

In [22]:
print(info)

[<div class="ellipsis rank01"><span>
<a href="javascript:melon.play.playSong('19041301',32872978);" title="Dynamite 재생">Dynamite</a>
</span></div>, <div class="ellipsis rank02">
<a href="javascript:melon.link.goArtistDetail('672375');" title="방탄소년단 - 페이지 이동">방탄소년단</a><span class="checkEllipsis" style="display:none"><a href="javascript:melon.link.goArtistDetail('672375');" title="방탄소년단 - 페이지 이동">방탄소년단</a></span>
</div>, <div class="ellipsis rank03">
<a href="javascript:melon.link.goAlbumDetail('10479150');" title="Dynamite (DayTime Version) - 페이지 이동">Dynamite (DayTime Version)</a>
</div>]


### Data Frame 만들기

In [13]:
ranks=[]; diffs =[]
titles=[]; singers = []; albums = []
service_list = soup.select_one('.service_list_song')
tbody = service_list.find('tbody')
trs = tbody.find_all('tr')

In [15]:
for tr in trs:
    rank = int(tr.select_one('.rank').string)
    
    info = tr.select_one('.ellipsis.rank01')
    title = info.find('a').string
    info = tr.select_one('.ellipsis.rank02')
    singer = info.find('a').string
    info = tr.select_one('.ellipsis.rank03')
    album = info.find('a').string

    entry = tr.select_one('.rank_wrap')
    spans = entry.find_all('span')
    diff = -100
    if len(spans) == 3:
        diff = int(spans[2].string)
        if diff != 0:
            sign = spans[2].attrs['class']
            if sign[0] == 'down':
                diff *= -1
    
    ranks.append(rank)
    diffs.append(diff)
    titles.append(title)
    singers.append(singer)
    albums.append(album)

In [16]:
top100 = pd.DataFrame({
    'rank': ranks,
    'diff': diffs,
    'title': titles,
    'singer': singers,
    'album': albums
})
top100 = top100.set_index('rank')
print(f'기간: {start_day}~{end_day}')
top100.head()

기간: 2020.11.09~2020.11.15


,diff,title,singer,album
rank,,,,
1,0,Dynamite,방탄소년단,Dynamite (DayTime Version)
2,1,힘든 건 사랑이 아니다,임창정,힘든 건 사랑이 아니다
3,-1,Lovesick Girls,BLACKPINK,THE ALBUM
4,1,취기를 빌려 (취향저격 그녀 X 산들),산들,취기를 빌려 (취향저격 그녀 X 산들)
5,-1,DON'T TOUCH ME,환불원정대,DON'T TOUCH ME
